In [1]:
# all libraries here
import sqlite3
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sklearn.preprocessing import OneHotEncoder
from scipy import stats
import matplotlib.pyplot as plt

from sklearn.feature_selection import mutual_info_regression
from sklearn.preprocessing import StandardScaler

In [2]:
conn = sqlite3.connect('data/score.db')
query = "SELECT * FROM score"

df = pd.read_sql(query, conn)
conn.close()

In [3]:
print(df.info())
print(df.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15900 entries, 0 to 15899
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   index               15900 non-null  int64  
 1   number_of_siblings  15900 non-null  int64  
 2   direct_admission    15900 non-null  object 
 3   CCA                 15900 non-null  object 
 4   learning_style      15900 non-null  object 
 5   student_id          15900 non-null  object 
 6   gender              15900 non-null  object 
 7   tuition             15900 non-null  object 
 8   final_test          15405 non-null  float64
 9   n_male              15900 non-null  float64
 10  n_female            15900 non-null  float64
 11  age                 15900 non-null  float64
 12  hours_per_week      15900 non-null  float64
 13  attendance_rate     15122 non-null  float64
 14  sleep_time          15900 non-null  object 
 15  wake_time           15900 non-null  object 
 16  mode

In [4]:
print(df.head(20))

    index  number_of_siblings direct_admission     CCA learning_style  \
0       0                   0              Yes  Sports         Visual   
1       1                   2               No  Sports       Auditory   
2       2                   0              Yes    None         Visual   
3       3                   1               No   Clubs       Auditory   
4       4                   0               No  Sports       Auditory   
5       5                   0               No    Arts         Visual   
6       6                   2              Yes    None         Visual   
7       7                   0               No  Sports         Visual   
8       8                   0               No    Arts       Auditory   
9       9                   2               No    Arts       Auditory   
10     10                   2              Yes    ARTS         Visual   
11     11                   2               No  Sports       Auditory   
12     12                   0               No  Spo

In [5]:
df = df.drop(['index', 'student_id', 'bag_color'], axis=1)

In [6]:
missing_values = df.isnull().sum()
missing_percentage = 100 * df.isnull().sum() / len(df)
missing_table = pd.concat([missing_values, missing_percentage], axis=1, keys=['Missing Count', 'Missing Percentage'])

print(missing_table)

                    Missing Count  Missing Percentage
number_of_siblings              0            0.000000
direct_admission                0            0.000000
CCA                             0            0.000000
learning_style                  0            0.000000
gender                          0            0.000000
tuition                         0            0.000000
final_test                    495            3.113208
n_male                          0            0.000000
n_female                        0            0.000000
age                             0            0.000000
hours_per_week                  0            0.000000
attendance_rate               778            4.893082
sleep_time                      0            0.000000
wake_time                       0            0.000000
mode_of_transport               0            0.000000


In [7]:
print(df['final_test'].describe())
print('\n')
print(df['attendance_rate'].describe())

count    15405.000000
mean        67.165401
std         13.977879
min         32.000000
25%         56.000000
50%         68.000000
75%         78.000000
max        100.000000
Name: final_test, dtype: float64


count    15122.000000
mean        93.270268
std          7.984230
min         40.000000
25%         92.000000
50%         95.000000
75%         97.000000
max        100.000000
Name: attendance_rate, dtype: float64


In [8]:
# multiple imputation looking way too complicated at the moment

In [9]:
# calculating the median
final_test_median = df['final_test'].median()
attendance_rate_median = df['attendance_rate'].median()

df['final_test'] = df['final_test'].fillna(final_test_median)
df['attendance_rate'] = df['attendance_rate'].fillna(attendance_rate_median)

missing_count = df.isnull().sum()
print(missing_count)

number_of_siblings    0
direct_admission      0
CCA                   0
learning_style        0
gender                0
tuition               0
final_test            0
n_male                0
n_female              0
age                   0
hours_per_week        0
attendance_rate       0
sleep_time            0
wake_time             0
mode_of_transport     0
dtype: int64


In [10]:
print(df.info())
print(df.head(20))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15900 entries, 0 to 15899
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   number_of_siblings  15900 non-null  int64  
 1   direct_admission    15900 non-null  object 
 2   CCA                 15900 non-null  object 
 3   learning_style      15900 non-null  object 
 4   gender              15900 non-null  object 
 5   tuition             15900 non-null  object 
 6   final_test          15900 non-null  float64
 7   n_male              15900 non-null  float64
 8   n_female            15900 non-null  float64
 9   age                 15900 non-null  float64
 10  hours_per_week      15900 non-null  float64
 11  attendance_rate     15900 non-null  float64
 12  sleep_time          15900 non-null  object 
 13  wake_time           15900 non-null  object 
 14  mode_of_transport   15900 non-null  object 
dtypes: float64(6), int64(1), object(8)
memory usage: 1.8+

In [11]:
print(df['sleep_time'].unique())
print(df['wake_time'].unique())

['22:00' '22:30' '21:00' '21:30' '23:00' '23:30' '1:00' '0:30' '1:30'
 '2:30' '0:00' '2:00' '3:00']
['6:00' '6:30' '5:00' '5:30' '7:00']


In [12]:
# converting object dtype (normally stored as strings) > datetime
df['sleep_time'] = pd.to_datetime(df['sleep_time'], format='%H:%M').dt.time
df['wake_time'] = pd.to_datetime(df['wake_time'], format='%H:%M').dt.time

# creating function to calculate sleep duration
def calculate_sleep_duration(sleep_time, wake_time):
    sleep_datetime = datetime.combine(datetime.min, sleep_time)
    wake_datetime = datetime.combine(datetime.min, wake_time)

    if sleep_time > wake_time:
        wake_datetime += timedelta(days=1)

    duration = wake_datetime - sleep_datetime
    return duration.total_seconds() / 3600 
# 3600 in an hour (60mins * 60secs)

df['sleep_duration'] = df.apply(lambda row: calculate_sleep_duration(row['sleep_time'], row['wake_time']), axis=1)
print(df[['sleep_time', 'wake_time', 'sleep_duration']])

      sleep_time wake_time  sleep_duration
0       22:00:00  06:00:00             8.0
1       22:30:00  06:30:00             8.0
2       22:30:00  06:30:00             8.0
3       21:00:00  05:00:00             8.0
4       21:30:00  05:30:00             8.0
...          ...       ...             ...
15895   22:00:00  06:00:00             8.0
15896   22:30:00  06:30:00             8.0
15897   23:00:00  07:00:00             8.0
15898   23:00:00  07:00:00             8.0
15899   23:00:00  07:00:00             8.0

[15900 rows x 3 columns]


In [13]:
print(df.info())
print(df.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15900 entries, 0 to 15899
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   number_of_siblings  15900 non-null  int64  
 1   direct_admission    15900 non-null  object 
 2   CCA                 15900 non-null  object 
 3   learning_style      15900 non-null  object 
 4   gender              15900 non-null  object 
 5   tuition             15900 non-null  object 
 6   final_test          15900 non-null  float64
 7   n_male              15900 non-null  float64
 8   n_female            15900 non-null  float64
 9   age                 15900 non-null  float64
 10  hours_per_week      15900 non-null  float64
 11  attendance_rate     15900 non-null  float64
 12  sleep_time          15900 non-null  object 
 13  wake_time           15900 non-null  object 
 14  mode_of_transport   15900 non-null  object 
 15  sleep_duration      15900 non-null  float64
dtypes: f

In [14]:
# encoding
# checking unique values for categorical columns

print("1. direct_admission:")
print(df['direct_admission'].unique())
print("\n2. CCA:")
print(df['CCA'].unique())
print("\n3. learning_style:")
print(df['learning_style'].unique())
print("\n4. gender:")
print(df['gender'].unique())
print("\n5. tuition:")
print(df['tuition'].unique())
print("\n6. mode_of_transport:")
print(df['mode_of_transport'].unique())

1. direct_admission:
['Yes' 'No']

2. CCA:
['Sports' 'None' 'Clubs' 'Arts' 'ARTS' 'SPORTS' 'CLUBS' 'NONE']

3. learning_style:
['Visual' 'Auditory']

4. gender:
['Female' 'Male']

5. tuition:
['No' 'Yes' 'Y' 'N']

6. mode_of_transport:
['private transport' 'public transport' 'walk']


In [15]:
# standardizing CCA column
def standardize_column(column):
    column = column.str.lower()
    unique_values = column.unique()
    value_map = {value: value for value in unique_values}
    return column.map(value_map)

df['CCA'] = standardize_column(df['CCA'])

print(df['CCA'].unique())

['sports' 'none' 'clubs' 'arts']


In [16]:
# standardizing tuition column
def standardize_tuition(column):
    column = column.str.lower()

    value_map = {
        'yes': 'yes',
        'y': 'yes', 
        'no': 'no',
        'n': 'no'
    }
    
    return column.map(value_map)

df['tuition'] = standardize_tuition(df['tuition'])
print(df['tuition'].value_counts())

tuition
yes    8996
no     6904
Name: count, dtype: int64


In [17]:
# encoding
columns_to_encode = ['direct_admission', 'CCA', 'learning_style', 'gender', 'tuition', 'mode_of_transport']

# initializing the onehotencoder
one_hot_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# fit and transform the selected columns
encoded_columns = one_hot_encoder.fit_transform(df[columns_to_encode])

# getting feature names
feature_names = one_hot_encoder.get_feature_names_out(columns_to_encode)

# creating new df with encoded columns, combining it with original df
encoded_df = pd.DataFrame(encoded_columns, columns=feature_names, index=df.index)
df = pd.concat([df.drop(columns=columns_to_encode), encoded_df], axis=1)

print(df.columns)

Index(['number_of_siblings', 'final_test', 'n_male', 'n_female', 'age',
       'hours_per_week', 'attendance_rate', 'sleep_time', 'wake_time',
       'sleep_duration', 'direct_admission_No', 'direct_admission_Yes',
       'CCA_arts', 'CCA_clubs', 'CCA_none', 'CCA_sports',
       'learning_style_Auditory', 'learning_style_Visual', 'gender_Female',
       'gender_Male', 'tuition_no', 'tuition_yes',
       'mode_of_transport_private transport',
       'mode_of_transport_public transport', 'mode_of_transport_walk'],
      dtype='object')


In [18]:
print(df.dtypes)

number_of_siblings                       int64
final_test                             float64
n_male                                 float64
n_female                               float64
age                                    float64
hours_per_week                         float64
attendance_rate                        float64
sleep_time                              object
wake_time                               object
sleep_duration                         float64
direct_admission_No                    float64
direct_admission_Yes                   float64
CCA_arts                               float64
CCA_clubs                              float64
CCA_none                               float64
CCA_sports                             float64
learning_style_Auditory                float64
learning_style_Visual                  float64
gender_Female                          float64
gender_Male                            float64
tuition_no                             float64
tuition_yes  

In [19]:
# converting sleep_time, wake_time to datetime
df['sleep_time'] = pd.to_datetime(df['sleep_time'], format='%H:%M:%S')
df['wake_time'] = pd.to_datetime(df['wake_time'], format='%H:%M:%S')

# Hour of day (cyclical encoding)
# good for lin reg, which assumes linear relationships, also tree-based models like RF, GBM
df['sleep_hour_sin'] = np.sin(df['sleep_time'].dt.hour * (2 * np.pi / 24))
df['sleep_hour_cos'] = np.cos(df['sleep_time'].dt.hour * (2 * np.pi / 24))
df['wake_hour_sin'] = np.sin(df['wake_time'].dt.hour * (2 * np.pi / 24))
df['wake_hour_cos'] = np.cos(df['wake_time'].dt.hour * (2 * np.pi / 24))

print(df.head(20))
print(df.dtypes)

    number_of_siblings  final_test  n_male  n_female   age  hours_per_week  \
0                    0        69.0    14.0       2.0  16.0            10.0   
1                    2        47.0     4.0      19.0  16.0             7.0   
2                    0        85.0    14.0       2.0  15.0             8.0   
3                    1        64.0     2.0      20.0  15.0            18.0   
4                    0        66.0    24.0       3.0  16.0             7.0   
5                    0        57.0     9.0      12.0  15.0            11.0   
6                    2        69.0    12.0       3.0  16.0            15.0   
7                    0        76.0    20.0       2.0  15.0             3.0   
8                    0        57.0    20.0       7.0  15.0            15.0   
9                    2        60.0    13.0       9.0  16.0            16.0   
10                   2        72.0    20.0       0.0  16.0            10.0   
11                   2        52.0    15.0       7.0  16.0      

In [20]:
df = df.drop(['sleep_time', 'wake_time'], axis=1)

In [21]:
# outliers
numerical_columns = ['number_of_siblings', 'final_test', 'n_male', 'n_female', 'age', 
                     'hours_per_week', 'attendance_rate', 'sleep_duration']

def detect_outliers_zscore(data, threshold=3):
    z_scores = np.abs(stats.zscore(data))
    return z_scores > threshold

def detect_outliers_iqr(data):
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 - 1.5 * IQR
    return (data < lower_bound) | (data > upper_bound)

outliers_zscore = {}
for col in numerical_columns:
    outliers_zscore[col] = detect_outliers_zscore(df[col])

outliers_iqr = {}
for col in numerical_columns:
    outliers_iqr[col] = detect_outliers_iqr(df[col])

print("Outliers detected using Z-score method:")
for col in numerical_columns:
    outlier_count = outliers_zscore[col].sum()
    print(f"{col}: {outlier_count} outliers ({outlier_count/len(df)*100:.2f}%)")

print("\nOutliers detected using IQR method:")
for col in numerical_columns:
    outlier_count = outliers_iqr[col].sum()
    print(f"{col}: {outlier_count} outliers ({outlier_count/len(df)*100:.2f}%)")

Outliers detected using Z-score method:
number_of_siblings: 0 outliers (0.00%)
final_test: 0 outliers (0.00%)
n_male: 0 outliers (0.00%)
n_female: 70 outliers (0.44%)
age: 451 outliers (2.84%)
hours_per_week: 0 outliers (0.00%)
attendance_rate: 537 outliers (3.38%)
sleep_duration: 625 outliers (3.93%)

Outliers detected using IQR method:
number_of_siblings: 15900 outliers (100.00%)
final_test: 15009 outliers (94.40%)
n_male: 13518 outliers (85.02%)
n_female: 15900 outliers (100.00%)
age: 15900 outliers (100.00%)
hours_per_week: 15418 outliers (96.97%)
attendance_rate: 15781 outliers (99.25%)
sleep_duration: 1279 outliers (8.04%)


In [22]:
print("age:")
print(df['age'].unique())
print("\nattendance_rate:")
print(df['attendance_rate'].unique())
print("\nsleep_duration:")
print(df['sleep_duration'].unique())

age:
[16. 15.  6.  5. -4. -5.]

attendance_rate:
[ 91.  94.  92.  95.  96.  93.  97.  98.  90. 100.  81.  87.  55.  64.
  99.  74.  61.  77.  52.  51.  56.  59.  67.  89.  73.  85.  82.  78.
  43.  54.  79.  72.  86.  63.  84.  70.  71.  45.  76.  60.  58.  88.
  66.  53.  80.  65.  69.  62.  57.  48.  83.  68.  44.  46.  50.  49.
  40.  75.  47.  42.  41.]

sleep_duration:
[8. 7. 5. 6. 4.]


In [23]:
# remove 6, 5, -4, -5 since its not possible for secondary school 
mode_age = df['age'].mode().iloc[0]
df['age'] = df['age'].replace([6, 5, -4, -5], mode_age)

print(df['age'].unique())

[16. 15.]


In [24]:
# selecting features to scale
print(df.head(20))
print(df.dtypes)

    number_of_siblings  final_test  n_male  n_female   age  hours_per_week  \
0                    0        69.0    14.0       2.0  16.0            10.0   
1                    2        47.0     4.0      19.0  16.0             7.0   
2                    0        85.0    14.0       2.0  15.0             8.0   
3                    1        64.0     2.0      20.0  15.0            18.0   
4                    0        66.0    24.0       3.0  16.0             7.0   
5                    0        57.0     9.0      12.0  15.0            11.0   
6                    2        69.0    12.0       3.0  16.0            15.0   
7                    0        76.0    20.0       2.0  15.0             3.0   
8                    0        57.0    20.0       7.0  15.0            15.0   
9                    2        60.0    13.0       9.0  16.0            16.0   
10                   2        72.0    20.0       0.0  16.0            10.0   
11                   2        52.0    15.0       7.0  16.0      

In [25]:
columns_to_scale = ['final_test', 'n_male', 'n_female', 'age', 'hours_per_week', 
                    'attendance_rate', 'number_of_siblings', 'sleep_duration']

scaler = StandardScaler()
df[columns_to_scale] = scaler.fit_transform(df[columns_to_scale])

print(df.head(20))

    number_of_siblings  final_test    n_male  n_female       age  \
0            -1.179974    0.131451  0.018314 -1.036376  1.028973   
1             1.481999   -1.467515 -1.507850  1.514782  1.028973   
2            -1.179974    1.294335  0.018314 -1.036376 -0.971843   
3             0.151013   -0.231951 -1.813082  1.664850 -0.971843   
4            -1.179974   -0.086590  1.544478 -0.886308  1.028973   
5            -1.179974   -0.740712 -0.744768  0.464305 -0.971843   
6             1.481999    0.131451 -0.286919 -0.886308  1.028973   
7            -1.179974    0.640212  0.934012 -1.036376 -0.971843   
8            -1.179974   -0.740712  0.934012 -0.286035 -0.971843   
9             1.481999   -0.522672 -0.134302  0.014101  1.028973   
10            1.481999    0.349491  0.934012 -1.336512  1.028973   
11            1.481999   -1.104114  0.170930 -0.286035  1.028973   
12           -1.179974    1.294335  0.018314 -0.736240  1.028973   
13            1.481999    1.148974  0.476163 -1.

In [26]:
# deciding which columns to drop with correlation analysis
# select all columns except for 'final_test'
feature_columns = [col for col in df.columns if col != 'final_test']

# calculate correlations with 'final_test'
correlations_finaltest = df[feature_columns].corrwith(df['final_test'])

# sort correlations by absolute value
correlations_sorted = correlations_finaltest.abs().sort_values(ascending=False)

print("Features correlations with final_test:")
print(correlations_sorted)

Features correlations with final_test:
CCA_none                               0.389343
number_of_siblings                     0.358453
attendance_rate                        0.334937
sleep_duration                         0.322292
learning_style_Visual                  0.268026
learning_style_Auditory                0.268026
tuition_no                             0.265320
tuition_yes                            0.265320
direct_admission_Yes                   0.236666
direct_admission_No                    0.236666
sleep_hour_sin                         0.212077
n_female                               0.169174
n_male                                 0.146674
hours_per_week                         0.145954
CCA_clubs                              0.137056
CCA_sports                             0.126682
CCA_arts                               0.124852
sleep_hour_cos                         0.094957
gender_Male                            0.011528
gender_Female                          0.011528
a

In [27]:
corr_matrix = df.corr()

def print_high_correlations(corr_matrix):
    for i in range(len(corr_matrix.columns)):
        for j in range(i+1, len(corr_matrix.columns)):
            if abs(corr_matrix.iloc[i,j]) >= 0.7 and abs(corr_matrix.iloc[i,j]) < 1:
                print(f"High correlation (>= 0.7) between {corr_matrix.index[i]} and {corr_matrix.columns[j]}: {corr_matrix.iloc[i,j]:.4f}")
            elif abs(corr_matrix.iloc[i,j]) == 1 and i != j:
                print(f"Perfect correlation (1.0) between {corr_matrix.index[i]} and {corr_matrix.columns[j]}: {corr_matrix.iloc[i,j]:.4f}")

# Call the function
print_high_correlations(corr_matrix)

# how to make it largest to smallest

High correlation (>= 0.7) between n_male and n_female: -0.7954
High correlation (>= 0.7) between attendance_rate and sleep_duration: 0.8679
High correlation (>= 0.7) between direct_admission_No and direct_admission_Yes: -1.0000
Perfect correlation (1.0) between learning_style_Auditory and learning_style_Visual: -1.0000
High correlation (>= 0.7) between gender_Female and gender_Male: -1.0000
High correlation (>= 0.7) between tuition_no and tuition_yes: -1.0000
High correlation (>= 0.7) between mode_of_transport_private transport and wake_hour_sin: 1.0000
High correlation (>= 0.7) between mode_of_transport_public transport and sleep_hour_cos: -0.8439
High correlation (>= 0.7) between mode_of_transport_public transport and wake_hour_cos: 0.8731
High correlation (>= 0.7) between mode_of_transport_walk and wake_hour_cos: -0.8033
High correlation (>= 0.7) between sleep_hour_sin and sleep_hour_cos: 0.8388
High correlation (>= 0.7) between sleep_hour_sin and wake_hour_cos: -0.7481
High correla

In [55]:
columns_to_drop = ['direct_admission_No', 'learning_style_Auditory', 'gender_Male', 
                   'tuition_no', 'wake_hour_sin']

df.drop(columns=columns_to_drop, inplace=True)
print(df.columns)

Index(['number_of_siblings', 'final_test', 'n_male', 'n_female', 'age',
       'hours_per_week', 'attendance_rate', 'sleep_duration',
       'direct_admission_Yes', 'CCA_arts', 'CCA_clubs', 'CCA_none',
       'CCA_sports', 'learning_style_Visual', 'gender_Female', 'tuition_yes',
       'mode_of_transport_private transport',
       'mode_of_transport_public transport', 'mode_of_transport_walk',
       'sleep_hour_sin', 'sleep_hour_cos', 'wake_hour_cos'],
      dtype='object')
